## Build Four-Junction Flux Qubit

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict

design = designs.DesignPlanar()
gui = MetalGUI(design)

# If you disable the next line, then you will need to delete a component [<component>.delete()] before recreating it.
design.overwrite_enabled = True

ERROR 1: PROJ: proj_create_from_database: Open of /home/daniel/anaconda3/envs/qiskit-metal/share/proj failed
  warnings.warn('Selected binding "{}" could not be found, '



In [3]:
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket

options = dict(
    pad_width = '425 um',
    pocket_height = '650um',
    connection_pads=dict(  # pin connectors
        a = dict(loc_W=+1,loc_H=-1),
        b = dict(loc_W=-1,loc_H=1),
        c = dict(loc_W=1,loc_H=1),
    )
)

q0 = TransmonPocket(design, 'Q0', options=dict(pos_x='-1.0mm', pos_y='-1.0mm', **options))
q1 = TransmonPocket(design, 'Q1', options=dict(pos_x='-1.0mm', pos_y='+0.0mm', **options))
q2 = TransmonPocket(design, 'Q2', options=dict(pos_x='1.0mm', pos_y='0.0mm', **options))
q2 = TransmonPocket(design, 'Q3', options=dict(pos_x='1.0mm', pos_y='-1.0mm', **options))

gui.rebuild()
gui.autoscale()

In [4]:
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
RoutePathfinder.get_template_options(design)

{'chip': 'main',
 'layer': '1',
 'pin_inputs': {'start_pin': {'component': '', 'pin': ''},
  'end_pin': {'component': '', 'pin': ''}},
 'fillet': '0',
 'lead': {'start_straight': '0mm',
  'end_straight': '0mm',
  'start_jogged_extension': '',
  'end_jogged_extension': ''},
 'total_length': '7mm',
 'trace_width': 'cpw_width',
 'anchors': {},
 'advanced': {'avoid_collision': 'true'},
 'step_size': '0.25mm',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False}

In [5]:
ops=dict(fillet='90um')

In [9]:
base_line_options = {
    'lead': {
        'start_straight': '91um',
        'end_straight': '90um'
    },
    'step_size': '0.25mm',
    **ops
}

options_01 = base_line_options.copy()
options_01['pin_inputs'] = {
    'start_pin': {'component': 'Q0', 'pin': 'a'},
    'end_pin': {'component': 'Q1', 'pin': 'b'}
}

options_12 = base_line_options.copy()
options_12['pin_inputs'] = {
    'start_pin': {'component': 'Q1', 'pin': 'a'},
    'end_pin': {'component': 'Q2', 'pin': 'b'}
}

options_23 = base_line_options.copy()
options_23['pin_inputs'] = {
    'start_pin': {'component': 'Q2', 'pin': 'a'},
    'end_pin': {'component': 'Q3', 'pin': 'b'}
}

options_30 = base_line_options.copy()
options_30['pin_inputs'] = {
    'start_pin': {'component': 'Q3', 'pin': 'a'},
    'end_pin': {'component': 'Q0', 'pin': 'b'}
}

design.delete_component('line01')
design.delete_component('line12')
design.delete_component('line23')
design.delete_component('line30')
RoutePathfinder(design, 'line01', options_01)
RoutePathfinder(design, 'line12', options_12)
RoutePathfinder(design, 'line23', options_23)
RoutePathfinder(design, 'line30', options_30)

gui.rebuild()
gui.autoscale()

## Capcitance Analysis

In [7]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [8]:
c1.sim.run(components=['Q0', 'Q1', 'Q2', 'Q3'])
c1.sim.capacitance_matrix

NameError: name 'Dispatch' is not defined